# General relativity using symbolic computer algebra

The purpose of this notebook is not to demonstrate capabilities of symbolic computer algebra. We rather would like to explore possibilities to visualize the Schwarzschild solution previously obtained by symbolic computation. The use of the `k3d` package allows us to rotate the figure and zoom in and out.

In [ ]:
import numpy as np
from scipy import integrate
import k3d

## Embedding of the Schwarzschild solution

Due to its high symmetry, the Schwarzschild lends itself to a graphical representation by embedding into a three-dimensional space. We will consider two different embeddings:

* The Schwarzschild space-time is static so that we can keep the time fixed. Then, we can visualize the spatial curvature properties.
* The second embedding makes use of the rotational symmetry and focusses on the radial coordinate and time. 

In the second point, we follow D. Marolf, *Spacetime Embedding Diagrams for Black Holes*, Gen. Rel. Grav. **31**, 919 (1999).

As derived in the previous noteboook [Schwarzschild solution](SchwarzschildMetric.ipynb), the Schwarzschild line element is given by
$$\mathrm{d}s^2 = -\left(1-\frac{r_\text{S}}{r}\right)c^2\mathrm{d}t^2+\frac{1}{1-\frac{r_\text{S}}{r}}\mathrm{d}r^2+r^2\mathrm{d}\theta^2+r^2\sin(\theta)^2\mathrm{d}\phi^2$$
where $r_\text{S}=2GM/c^2$ is the Schwarzschild radius.

## Spatial properties

The stationarity of the Schwarzschild solution allows us to fix the time coordinate $t$. Furthermore, due to the rotational symmetry, we can restrict ourselves to the equatorial plane $\theta=\pi/2$. We thus are left with the two coordinates $r$ and $\phi$ and the line element
$$\mathrm{d}s^2 = \frac{1}{1-\frac{r_\text{S}}{r}}\mathrm{d}r^2+r^2\mathrm{d}\phi^2.$$

Our goal is to determine an embedding in a three-dimensional space. To account for the axial symmetry, we choose cylindrical coordinates $(r,\phi,z$ with the line element
$$\mathrm{d}s^2 = \mathrm{d}r^2+r^2\mathrm{d}\phi^2+\mathrm{d}z^2$$
representing a three-dimensional flat space. We will embed the Schwarzschild metric by choosing an appropriate function $z(r)$ defining a two-dimensional surface. On this surface,
the line element is given by
$$\mathrm{d}s^2 = \left[1+\left(\frac{\mathrm{d}z}{\mathrm{d}r}\right)^2\right]\mathrm{d}r^2+r^2\mathrm{d}\phi^2.$$

Comparing with the Schwarzschild metric, we obtain the differential equation
$$\frac{\mathrm{d}z}{\mathrm{d}r} = \pm\sqrt{\frac{1}{1-\frac{r_\text{S}}{r}}-1}
=\pm\sqrt{\frac{r_\text{S}}{r-r_\text{S}}}.$$
In the following, we choose the positive branch of the square root.

The integration is straightforward and we obtain
$$z(r) = 2\sqrt{(r-r_\text{S})r_\text{S}}.$$
In the following, all lengths will be taken in units of the Schwarzschild radius.

In [ ]:
def line_r(r):
    """generate line for constant r"""
    phi = np.linspace(0, 2*np.pi, 100, dtype=np.float32)
    x = r*np.cos(phi)
    y = r*np.sin(phi)
    z = 2*np.sqrt(r-1)*np.ones_like(phi)
    return k3d.line(np.vstack([x, y, z]).T)

def line_phi(phi, rmax):
    """generate line for constant phi"""
    r = np.linspace(1, rmax, 100, dtype=np.float32)
    x = r*np.cos(phi)
    y = r*np.sin(phi)
    z = 2*np.sqrt(r-1)
    return k3d.line(np.vstack([x, y, z]).T)

plot = k3d.plot()
rmax = 5
for r in np.linspace(1, rmax, 20):
    plot += line_r(r)
for phi in np.linspace(0, 2*np.pi, 40):
    plot += line_phi(phi, rmax)
plot.display()

The lines in this figure have been drawn for equidistant values of the radial coordinate $r$ starting at $r=r_\text{S}$. For a given value of $r$, the circumference of the corresponding circle equals $2\pi r$. On the other hand, as the figure clearly shows, the radial distance does not correspond to the difference of the corresponding values of $r$. Close to the Schwarzschild radius at the bottom of the figure, the radial distance between successive values of $r$ is bigger than further out. Rotate and zoom the figure to explore in more detail this property.

## Space-time properties

We now consider the Schwarzschild metric at a fixed value of the angles $\theta$ and $\phi$, keeping the time $t$ and the radial coordinate $r$ as variables. The Schwarzschild solution will be embedded into a (1+2)-dimensional Minkowski space with line element
$$\mathrm{d}s^2 = -c^2\mathrm{d}T^2+\mathrm{d}X^2+\mathrm{d}Y^2.$$
The embedding is achieved by setting
\begin{align}
 cT &= 2r_\text{S}\sqrt{1-\dfrac{r_\text{S}}{r}}\sinh\left(\dfrac{ct}{2r_\text{S}}\right)\\
 X &= 2r_\text{S}\sqrt{1-\dfrac{r_\text{S}}{r}}\cosh\left(\dfrac{ct}{2r_\text{S}}\right)\\
\end{align}
and
$$Y(r)=-\int_{r_\text{S}}^r\sqrt{1+\dfrac{r_\text{S}}{r}+\dfrac{r_\text{S}^2}{r^2}
                                +\dfrac{r_\text{S}^3}{r^3}}.$$
The latter choice ensures that the line element on the embedded surface is given by the Schwarzschild line element at fixed angles $\theta$ and $\phi$
$$\mathrm{d}s^2 = -\left(1-\dfrac{r_\text{S}}{r}\right)c^2\mathrm{d}t^2
                  +\frac{1}{1-\dfrac{r_\text{S}}{r}}\mathrm{d}r^2.$$
                  
The following code produces a visual representation of this embedding. In interpreting the result, it should be kept in mind that we embedded into a (2+1)-dimensional Minkowski space, not a 3-dimensional Euclidean space.

A technical remark on the creation of the figure: There is some flexibility in choosing the values $(X, cT)$ for which $Y$ is calculated and which will be shown as point $(X, Y, cT)$ in the figure and denoted by `x, y, t` in the code. It turns out that instead of using $X$ directly, it is better to use a parameter `v` and define
$$x = \tanh\left(\frac{5}{2}v\right)\sqrt{v^2+t^2}.$$
The factor $5/2$ can be modified, but we follow here the choice made in the paper by Marolf cited above. Feel free to experiment with a parametrization based directly on $X$ and $cT$. Alternatively, take a look at the discussion in the paper by Marolf.

In [ ]:
def integrand(u):
    return -np.sqrt(1 + 1/u + 1/u**2 + 1/u**3)

def y_integral(r):
    """determine function Y(r) for embedding"""
    return integrate.quad(integrand, 1, r)[0]

def line_v(v, tmax):
    """generate line for constant v"""
    t = np.linspace(-tmax, tmax, 100, dtype=np.float32)
    x = np.tanh(2.5*v)*np.sqrt(v**2+t**2)
    r = 1/(1-0.25*(x**2-t**2))
    y = np.array([y_integral(r[n]) for n in range(r.shape[0])], dtype=np.float32)
    return k3d.line(np.vstack([x, y, t]).T)

def line_t(t, vmax):
    """generate line for constant t"""
    v = np.linspace(-vmax, vmax, 100, dtype=np.float32)
    x = np.tanh(2.5*v)*np.sqrt(v**2+t**2)
    r = 1/(1-0.25*(x**2-t**2))
    y = np.array([y_integral(r[n]) for n in range(r.shape[0])], dtype=np.float32)
    z = t*np.ones_like(r)
    return k3d.line(np.vstack([x, y, z]).T)

def line_r(r, tmax):
    """generate line for constant r
    
       r is taken in units of the Schwarzschild radius.
       
       We need to distinguish the regions larger than the Schwarzschild radius (r>1)
       and smaller than the Schwarzschild radius (r<1).
       
    """
    if r >= 1:
        t = np.linspace(-tmax, tmax, 100, dtype=np.float32)
        x = np.sqrt(t*t+4*(1-1/r))
        y = y_integral(r)*np.ones_like(t)
        return np.vstack([x, y, t]).T, np.vstack([-x, y, t]).T
    else:
        tmin = 2*np.sqrt(1/r-1)+1e-7
        t = np.linspace(tmin, tmax, 100, dtype=np.float32)
        x = np.sqrt(t*t+4*(1-1/r))
        y = y_integral(r)*np.ones_like(t)
        return (np.vstack([x, y, t]).T, np.vstack([-x, y, t]).T,
                np.vstack([x, y, -t]).T, np.vstack([-x, y, -t]).T)

plot = k3d.plot(axes=['X', 'Y', 'cT'])
vmax = 1.9
tmax = 10

# draw grid parametrized by v and t
for v in np.linspace(-vmax, vmax, 30):
    plot += line_v(v, tmax)
for t in np.linspace(-tmax, tmax, 30):
    plot += line_t(t, vmax)
    
# draw pair of lines indicating the Schwarzschild radius
kwargs = {'color': 0xc00000, 'width': 0.04}
plot += k3d.line([[-tmax, 0, -tmax], [tmax, 0, tmax]], **kwargs)
plot += k3d.line([[tmax, 0, -tmax], [-tmax, 0, tmax]], **kwargs)

# draw lines of constant r
kwargs = {'color': 0x008000, 'width': 0.04}
for r in (0.125, 0.25, 0.5, 2, 4, 8):
    for l in line_r(r, tmax):
        plot += k3d.line(l, **kwargs)
        
plot.display()

The red lines indicate the Schwarzschild radius corresponding to $Y=0$, separating the surface into four distinct regions. Positive values of $Y$ correspond to distances $r$ from the central mass larger than the Schwarzschild radius. The green lines represent curves of constant distance from the central mass with $r$ different from the Schwarzschild radius.